In [28]:
# Asegurar que las gráficas se muestren en línea en el Jupyter Notebook
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
# Importa la función para crear la matriz de confusión
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn import metrics # Para calcular métricas de rendimiento del modelo.
from sklearn import tree

#cr_cp = pm.df('cr_cp')
#fe_cp = pm.df('fe_cp')
df_jo = pm.df('df_jo')
#df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
#df_jo = df_jo.drop(columns=['index'])
#df_jo = df_jo.drop(columns=['Mes_created_at'])
#df_jo_cp = df_jo.copy()
df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = df_jo.drop(columns=['index'])
#df_jo.info()

In [29]:
df = df_jo.copy()

# Para utilizar modelos de árbol de decisión.
# Establecer la semilla para la aleatorización (asegura resultados repetibles).
np.random.seed(42)

# Convertir columnas datetime a timestamps
for col in df.select_dtypes(include=['datetime64']).columns:
    df[col] = df[col].apply(lambda x: x.timestamp() if pd.notnull(x) else None)

for col in df.select_dtypes(include=['timedelta64']).columns:
    df[col] = df[col].apply(lambda x: x / pd.to_timedelta(1, unit='d') if pd.notnull(x) else None)

# Variables predictoras (ajusta según los datos disponibles)
columnas = ['amount','fee','n_backs','n_fees','n_incidents','to_reimbur', 'type', 'transfer_type','category','charge_moment'] #'user_id',   
X = df[columnas].copy()

# Categoricas
X = pd.get_dummies(X, columns=['type', 'transfer_type', 'category','charge_moment'], drop_first=True, dtype =int)

# Crear el escalador
display(X.head(1))
scaler = StandardScaler()
# Estandarizar todas las columnas
normalizado = scaler.fit_transform(X)
# Convertir de nuevo a DataFrame, preservando nombres de columnas e índices
X = pd.DataFrame(normalizado, columns=X.columns, index=df.index)
X.fillna(0, inplace=True)
#X.info()
y = df['needs_m_check_recov'].copy()  #moderada # La columna de la variable objetivo

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

# Crea el clasificador de regresión logística. El 'liblinear' usa metodo de optimización de minimos cuadrados generalizados (L2) y soporta la regularización L1 y L2
clf = LogisticRegression(solver="liblinear")
clf.fit(X_train, y_train) # Entrena el clasificador
predicciones = clf.predict(X_test) # Realiza predicciones sobre el conjunto de prueba

# Genera las probabilidades de predicción
predicciones_probabilidades = clf.predict_proba(X_test)


,amount,fee,n_backs,n_fees,n_incidents,to_reimbur,type_incident,type_instant_payment,type_postpone,transfer_type_regular,category_nice,category_rejected_direct_debit,charge_moment_before
0,1.0,0.0,0,0,1,16.376464,0,0,0,1,0,0,0


In [ ]:
# Establecer la complejidad del árbol de decisión en 5 (profundidad máxima de 5).
C = 5 # Profundidad máxima del árbol de decisión (establecido en 5).
MAXN = 1000 # Número máximo de muestras por clase.

# Inicializar matrices para almacenar las tasas de error para 10 iteraciones y diferentes números de muestras de entrenamiento.
yhat_test_c5 = np.zeros((10, 299, 2)) # Error en prueba (tamaño:[iteraciones, tamaños de muestra, conjunto])
yhat_train_c5 = np.zeros((10, 299, 2)) # Error en entrenamiento

# Ejecutar el experimento 10 veces para obtener curvas suavizadas promedio.
for iteration in range(10):
    # Generar datos sintéticos para el conjunto de entrenamiento.
    # Usar distribuciones normales para crear datos representativos.
    X_train = np.concatenate([
        1.25 * np.random.randn(MAXN, 2), # Primera distribución
        5 + 1.5 * np.random.randn(MAXN, 2), # Segunda distribución
        [8, 5] + 1.5 * np.random.randn(MAXN, 2) # Tercera distribución
    ])
    y_train = np.concatenate([
        np.ones((MAXN, 1)), # Etiquetas para la primera clase
        -np.ones((MAXN, 1)), # Etiquetas para la segunda clase
        np.ones((MAXN, 1)) # Etiquetas para la tercera clase
    ])

    # Aleatorizar los datos para asegurar la variabilidad.
    perm = np.random.permutation(y_train.size)
    X_train = X_train[perm, :]
    y_train = y_train[perm]

    # Generar datos sintéticos para el conjunto de prueba.
    X_test = np.concatenate([
        1.25 * np.random.randn(MAXN, 2),
        5 + 1.5 * np.random.randn(MAXN, 2),
        [8, 5] + 1.5 * np.random.randn(MAXN, 2)
    ])
    y_test = np.concatenate([
        np.ones((MAXN, 1)),
        -np.ones((MAXN, 1)),
        np.ones((MAXN, 1))
    ])

    # Evaluar el modelo con diferentes tamaños de muestra.
    for j, N in enumerate(range(10, 3000, 10)):
        # Tomar un subconjunto de datos para entrenamiento con los primeros N ejemplos.
        X_subset = X_train[:N, :]
        y_subset = y_train[:N]
        # Crear y entrenar un clasificador de árbol de decisión con una profundidad máxima de C (ahora 5).
        clf = tree.DecisionTreeClassifier(min_samples_leaf=1, max_depth=C)
        clf.fit(X_subset, y_subset.ravel()) # Ajustar el modelo a los datos de entrenamiento.
        # Evaluar el modelo en el conjunto de prueba y calcular la tasa de error.
        yhat_test_c5[iteration, j, 1] = 1 -  metrics.accuracy_score(clf.predict(X_test), y_test.ravel())
        # Evaluar el modelo en el conjunto de entrenamiento y calcular la tasa de error.
        yhat_train_c5[iteration, j, 1] = 1 -  metrics.accuracy_score(clf.predict(X_subset), y_subset.ravel())

# Promediar las tasas de error a lo largo de las iteraciones para obtener una curva de aprendizaje más estable.
mean_test_error_c5 = np.mean(yhat_test_c5[:, :, 1].T, axis=1)
mean_train_error_c5 = np.mean(yhat_train_c5[:, :, 1].T, axis=1)

# Graficar las curvas de aprendizaje: tasa de error en prueba y entrenamiento.
plt.figure(figsize=(9, 5))
plt.plot(mean_test_error_c5, 'r', label='Error en Prueba') # Error en el conjunto de prueba.
plt.plot(mean_train_error_c5, 'b', label='Error en Entrenamiento') # Error en el conjunto de entrenamiento.

# Configurar etiquetas, título y leyenda.
plt.xlabel('Número de muestras x10', fontsize=12)
plt.ylabel('Tasa de Error', fontsize=12)
plt.title('Curvas de Aprendizaje para un Árbol de Decisión con Profundidad Máxima = 5', fontsize=14)
plt.legend(loc='upper left', fontsize=12, bbox_to_anchor=(1, 1))

# Mostrar la gráfica con las curvas de error.
plt.grid(True)
plt.tight_layout() # Ajustar la distribución para evitar que las etiquetas se corten.
plt.show()

InvalidIndexError: (slice(None, 10, None), slice(None, None, None))